In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import chromedriver_binary
from pymongo import MongoClient 
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
% matplotlib inline

In [2]:
# making connection to mongodb

client = MongoClient()
database_schoolFinds = client['db_schoolFinds']
collection_schoolsFindsPass = database_schoolFinds['collection_schoolFindsPass']
collection_schoolsFindsFail = database_schoolFinds['collection_schoolFindsFail']

In [6]:
driver = webdriver.Chrome()
driver.get("http://www.schoolfinds.com/")
driver.maximize_window()

# finding all the cities links

driver.find_element_by_tag_name('body').send_keys(Keys.HOME)
sleep(3)

selectCityButton = driver.find_element_by_xpath("//div[@class='header-content visible-desktop']//div[@class='span4']//a[2]")
selectCityButton.click()
sleep(2)

allCityLinks = driver.find_elements_by_xpath("//div[@class='modal-body']//ul[@class='last']//a")

closeButton = driver.find_element_by_xpath('//*[@id="popup"]/footer/button')
closeButton.click()
sleep(2)

In [7]:
all_city_links_list = []

for eachCity in allCityLinks:
    all_city_links_list.append(eachCity.get_attribute('href'))
    
    
all_city_links_list = all_city_links_list[28:100]

In [10]:
all_city_links_list

['http://www.schoolfinds.com/delhi/narela.html',
 'http://www.schoolfinds.com/delhi/bhajan-pura.html',
 'http://www.schoolfinds.com/delhi/new-delhi.html',
 'http://www.schoolfinds.com/delhi/dwarka.html',
 'http://www.schoolfinds.com/delhi/vasant-vihar.html',
 'http://www.schoolfinds.com/delhi/south-west-delhi.html',
 'http://www.schoolfinds.com/delhi/north-west-delhi.html',
 'http://www.schoolfinds.com/delhi/janak-puri.html',
 'http://www.schoolfinds.com/delhi/east-new-delhi.html',
 'http://www.schoolfinds.com/delhi/nangloi.html',
 'http://www.schoolfinds.com/delhi/lajpat-nagar.html',
 'http://www.schoolfinds.com/delhi/mehrauli.html',
 'http://www.schoolfinds.com/delhi/rohini.html',
 'http://www.schoolfinds.com/delhi/pitampura.html',
 'http://www.schoolfinds.com/delhi/south-delhi.html',
 'http://www.schoolfinds.com/delhi/shahdara.html',
 'http://www.schoolfinds.com/delhi/mayur-vihar.html',
 'http://www.schoolfinds.com/delhi/uttam-nagar.html',
 'http://www.schoolfinds.com/delhi/badarpur

In [11]:
for eachCity in all_city_links_list:
    
    driver.get(eachCity)
    sleep(5)
    
    noOfSchoolsTotal = int(re.sub(r"\D", '', str(driver.find_element_by_xpath(r"//div[@class='product-count']").text)))  # 196
    print()
    print("Total no of schools are : ", noOfSchoolsTotal)
    
    noOfOlsEstimated = int(noOfSchoolsTotal/10)   # 19 suppose
    print("possible number of ols estimated : ", noOfOlsEstimated)
    
    print("scrolling to page end!!! : for ",noOfOlsEstimated, " times")
    
    for i in range(noOfOlsEstimated):   # cover ols created
        
        driver.find_element_by_tag_name('body').send_keys(Keys.END)
        sleep(3)
        
        
        
    if (noOfSchoolsTotal < 10):
        print("no of schools are less than 10")
        
        noOfLisPresent_in_this_ol = len(driver.find_elements_by_xpath(r"//ol[@id='products-list'][1]" + "//li[contains(@class,'item')]"))
        print()
        print("No of lis  presentin this single ol :", noOfLisPresent_in_this_ol)
        
        for j in range(1, noOfLisPresent_in_this_ol + 1):  # for this "ol", specifically, go by individual "li" under this "ol"
            
            try:
                schoolAboutDict = {}

                schoolName = driver.find_element_by_xpath("//ol[@id='products-list'][" + str(1) + "]" + "//li[contains(@class,'item')][" + str(j) + "]//h2").text
                schoolAboutDict['schoolName'] = schoolName

                schoolAddress = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(1) + "]" +  "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span").text
                schoolAddressRefined = re.findall(r"Address(.*?)\nEmail", schoolAddress)[0]
                schoolAboutDict['schoolAddress'] = schoolAddressRefined

                schoolEmail = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(1) + "]" +  "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span//div[@class='std' and @itemprop='description'][1]").text
                schoolAboutDict['schoolEmail'] = schoolEmail

                schoolContact = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(1) + "]" +  "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span//div[@class='std' and @itemprop='description'][2]").text
                schoolAboutDict['schoolContact'] = schoolContact

                schoolURL = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(1) + "]" + "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span//div[@class='std' and @itemprop='description'][4]//a").get_attribute('href')
                schoolAboutDict['schoolURL'] = schoolURL
                
                if (collection_schoolsFindsPass.find_one({"schoolName":schoolName, "schoolAddress":schoolAddressRefined})):
                    print("Already Present in the db")
                    continue

                collection_schoolsFindsPass.insert_one(schoolAboutDict)
                sleep(2)
                
                print("School Details : ", schoolAboutDict)
                print()
                
            except:
                
                print("Error for the head url city : ", eachCity)
                
                schoolDictFail = {}
                schoolDictFail['Error City URL'] = str(eachCity)
                
                if len(str(schoolName) > 0):
                    schoolDictFail['Error School Name'] = str(schoolName)
                
                collection_schoolsFindsFail.insert_one(schoolDictFail)
                
                
        
    else:
        
        if (noOfSchoolsTotal > noOfOlsEstimated*10):   # 196 > 190 
            print("WE NEED AN EXTRAA ITERATION!!!!  AS  :", noOfSchoolsTotal," > " ,noOfOlsEstimated*10)
        
            for i in range(1, noOfOlsEstimated + 2):    # go by "ol" number 1,2,3,4,5,6,7,8,9,.....18,19,20

                print("-------------------------STARTING FOR OL---------------------------- : ", i)
                print()

                noOfLisPresent_in_this_ol = len(driver.find_elements_by_xpath(r"//ol[@id='products-list'][" + str(i) + "]" + "//li[contains(@class,'item')]"))
                print()
                print("No of lis  presentin this ol :", noOfLisPresent_in_this_ol)

                for j in range(1, noOfLisPresent_in_this_ol + 1):  # for this "ol", specifically, go by individual "li" under this "ol"

                    try:
                        schoolAboutDict = {}

                        schoolName = driver.find_element_by_xpath("//ol[@id='products-list'][" + str(i) + "]" + "//li[contains(@class,'item')][" + str(j) + "]//h2").text
                        schoolAboutDict['schoolName'] = schoolName

                        schoolAddress = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(i) + "]" +  "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span").text
                        schoolAddressRefined = re.findall(r"Address(.*?)\nEmail", schoolAddress)[0]
                        schoolAboutDict['schoolAddress'] = schoolAddressRefined

                        schoolEmail = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(i) + "]" +  "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span//div[@class='std' and @itemprop='description'][1]").text
                        schoolAboutDict['schoolEmail'] = schoolEmail

                        schoolContact = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(i) + "]" +  "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span//div[@class='std' and @itemprop='description'][2]").text
                        schoolAboutDict['schoolContact'] = schoolContact

                        schoolURL = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(i) + "]" + "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span//div[@class='std' and @itemprop='description'][4]//a").get_attribute('href')
                        schoolAboutDict['schoolURL'] = schoolURL

                        if (collection_schoolsFindsPass.find_one({"schoolName":schoolName, "schoolAddress":schoolAddressRefined})):
                            print("Already Present in the db")
                            continue


                        collection_schoolsFindsPass.insert_one(schoolAboutDict)
                        print("School Inserted with success")
                        sleep(2)

                        print("School Details : ", schoolAboutDict)
                        print()



                    except:

                        print("Error for the head url city : ", eachCity)

                        schoolDictFail = {}
                        schoolDictFail['Error City URL'] = str(eachCity)

                        if len(str(schoolName) > 0):
                            schoolDictFail['Error School Name'] = str(schoolName)

                        collection_schoolsFindsFail.insert_one(schoolDictFail)

        else:
            print("schools can be exact multiples of 10 ")
            
            for i in range(1, noOfOlsEstimated + 1):    # go by "ol" number 1,2,3,4,5,6,7,8,9,.....18,19

                print("-------------------------STARTING FOR OL---------------------------- : ", i)
                print()

                noOfLisPresent_in_this_ol = len(driver.find_elements_by_xpath(r"//ol[@id='products-list'][" + str(i) + "]" + "//li[contains(@class,'item')]"))
                print()
                print("No of lis  presentin this ol :", noOfLisPresent_in_this_ol)

                for j in range(1, noOfLisPresent_in_this_ol + 1):  # for this "ol", specifically, go by individual "li" under this "ol"

                    try:
                        schoolAboutDict = {}

                        schoolName = driver.find_element_by_xpath("//ol[@id='products-list'][" + str(i) + "]" + "//li[contains(@class,'item')][" + str(j) + "]//h2").text
                        schoolAboutDict['schoolName'] = schoolName

                        schoolAddress = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(i) + "]" +  "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span").text
                        schoolAddressRefined = re.findall(r"Address(.*?)\nEmail", schoolAddress)[0]
                        schoolAboutDict['schoolAddress'] = schoolAddressRefined

                        schoolEmail = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(i) + "]" +  "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span//div[@class='std' and @itemprop='description'][1]").text
                        schoolAboutDict['schoolEmail'] = schoolEmail

                        schoolContact = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(i) + "]" +  "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span//div[@class='std' and @itemprop='description'][2]").text
                        schoolAboutDict['schoolContact'] = schoolContact

                        schoolURL = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(i) + "]" + "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span//div[@class='std' and @itemprop='description'][4]//a").get_attribute('href')
                        schoolAboutDict['schoolURL'] = schoolURL

                        if (collection_schoolsFindsPass.find_one({"schoolName":schoolName, "schoolAddress":schoolAddressRefined})):
                            print("Already Present in the db")
                            continue


                        collection_schoolsFindsPass.insert_one(schoolAboutDict)
                        print("School Inserted with success")
                        sleep(2)

                        print("School Details : ", schoolAboutDict)
                        print()



                    except:

                        print("Error for the head url city : ", eachCity)

                        schoolDictFail = {}
                        schoolDictFail['Error City URL'] = str(eachCity)

                        if len(str(schoolName) > 0):
                            schoolDictFail['Error School Name'] = str(schoolName)

                        collection_schoolsFindsFail.insert_one(schoolDictFail)

        


Total no of schools are :  18
possible number of ols estimated :  1
scrolling to page end!!! : for  1  times
WE NEED AN EXTRAA ITERATION!!!!  AS  : 18  >  10
-------------------------STARTING FOR OL---------------------------- :  1


No of lis  presentin this ol : 10
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
-------------------------STARTING FOR OL---------------------------- :  2


No of lis  presentin this ol : 8
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db

Total no of schools are :  4
possible number of ols estimated :  0
scrolling to page end!!! : for  0  times
no of schools are less than 10

No of lis  pr

Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
-------------------------STARTING FOR OL---------------------------- :  20


No of lis  presentin this ol : 10
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
-------------------------STARTING FOR OL---------------------------- :  21


No of lis  presentin this ol : 10
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
-------------------------STARTING FOR OL---------------------------- :  22


No of lis  presentin this ol : 10
Already Present i

Already Present in the db
Already Present in the db
Already Present in the db
-------------------------STARTING FOR OL---------------------------- :  42


No of lis  presentin this ol : 10
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
-------------------------STARTING FOR OL---------------------------- :  43


No of lis  presentin this ol : 10
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
-------------------------STARTING FOR OL---------------------------- :  44


No of lis  presentin this ol : 10
Already Present in the db
Already Present in the db
Already Present i

Already Present in the db
-------------------------STARTING FOR OL---------------------------- :  64


No of lis  presentin this ol : 10
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
-------------------------STARTING FOR OL---------------------------- :  65


No of lis  presentin this ol : 10
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
-------------------------STARTING FOR OL---------------------------- :  66


No of lis  presentin this ol : 10
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present i


No of lis  presentin this ol : 10
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
-------------------------STARTING FOR OL---------------------------- :  87


No of lis  presentin this ol : 10
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
-------------------------STARTING FOR OL---------------------------- :  88


No of lis  presentin this ol : 10
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present

...

School Inserted with success
School Details :  {'schoolName': 'H D SR SEC PUBLIC SCHOOL', 'schoolAddress': ': Vill Kheri Meham Distt Rohtak Haryana -124112', 'schoolEmail': 'Email: hdpskherimeham@yahoo.com', 'schoolContact': 'Contact Number: 01257 -233381 234381', 'schoolURL': 'http://www.schoolfinds.com/h-d-sr-sec-public-school.html', '_id': ObjectId('5b89c4e8edf2ee1498acd43f')}

School Inserted with success
School Details :  {'schoolName': 'GURUKUL HIGH SCHOOL BHAIYAPUR LADHOUT', 'schoolAddress': ': bhaiyapur ladhout rohtak (haryana) 124001 -124001', 'schoolEmail': 'Email: gurukulbhaiyapur@yahoo.com', 'schoolContact': 'Contact Number: -01262-217550', 'schoolURL': 'http://www.schoolfinds.com/gurukul-high-school-bhaiyapur-ladhout.html', '_id': ObjectId('5b89c4ebedf2ee1498acd440')}

School Inserted with success
School Details :  {'schoolName': 'APEX PUBLIC SCHOOL', 'schoolAddress': ': KAHRAWAR DELHI ROAD (NEAR HANUMAN MANDIR) ROHTAK HARYANA -124001', 'schoolEmail': 'Email:', 'schoolCont

School Inserted with success
School Details :  {'schoolName': 'PARTH PUBLIC SCHOOL', 'schoolAddress': ': G T ROAD NEAR GOVT SR SEC SCHOOL GHARAUNDA KARNAL HARYANA -132001', 'schoolEmail': 'Email: skpps@sancharnet.in', 'schoolContact': 'Contact Number: 01748 -52833', 'schoolURL': 'http://www.schoolfinds.com/parth-public-school.html', '_id': ObjectId('5b89c545edf2ee1498acd453')}

School Inserted with success
School Details :  {'schoolName': 'OPS VIDYA MANDIR', 'schoolAddress': ': SECTOR 13 U E KARNAL HARYANA -132001', 'schoolEmail': 'Email: skool.ms.chkd@apj.edu', 'schoolContact': 'Contact Number: 01250 -243341', 'schoolURL': 'http://www.schoolfinds.com/ops-vidya-mandir.html', '_id': ObjectId('5b89c548edf2ee1498acd454')}

School Inserted with success
School Details :  {'schoolName': 'GURU TEG BAHADUR PUBLIC SCHOOL', 'schoolAddress': ': model town karnal haryana -132001', 'schoolEmail': 'Email: kiran_saini98@rediffmail.com', 'schoolContact': 'Contact Number: 1214000772', 'schoolURL': 'htt

School Inserted with success
School Details :  {'schoolName': 'M.D. SENIOR SECONDARY SCHOOL', 'schoolAddress': ': V.P.O. Staundi- Distt. Karnal- Haryana -132001', 'schoolEmail': 'Email: rjnlamba@yahoo.co.in', 'schoolContact': 'Contact Number: 04865 -01748-253266', 'schoolURL': 'http://www.schoolfinds.com/m-d-senior-secondary-school.html', '_id': ObjectId('5b89c584edf2ee1498acd469')}

-------------------------STARTING FOR OL---------------------------- :  6


No of lis  presentin this ol : 6
School Inserted with success
School Details :  {'schoolName': 'M N M PUBLIC SCHOOL', 'schoolAddress': ': KARNAL ASANDH ROAD JUNDLA DISTT KARNAL HARYANA -132036', 'schoolEmail': 'Email: awareness100@yahoo.com', 'schoolContact': 'Contact Number: 01745 -274905', 'schoolURL': 'http://www.schoolfinds.com/m-n-m-public-school.html', '_id': ObjectId('5b89c587edf2ee1498acd46a')}

School Inserted with success
School Details :  {'schoolName': 'KENDRIYA VIDYALAYA', 'schoolAddress': ': HARSINGH PURA DISTT KARNAL

School Inserted with success
School Details :  {'schoolName': 'ST JOSEPH SCHOOL', 'schoolAddress': ': 129 SHAHZADPUR HOUSE CIVIL LINES AMBALA CITY HARYANA -134003', 'schoolEmail': 'Email: hareram.1305@gmail.com', 'schoolContact': 'Contact Number: 0171 -2550329', 'schoolURL': 'http://www.schoolfinds.com/st-joseph-school.html', '_id': ObjectId('5b89c5f2edf2ee1498acd47c')}

School Inserted with success
School Details :  {'schoolName': 'SPRING FIELD PUBLIC SCHOOL', 'schoolAddress': ': AMBALA CHANDIGARH HIGHWAY AMBALA HARYANA -134007', 'schoolEmail': 'Email: spring77@rediffmail.com', 'schoolContact': 'Contact Number: 0484 -540663', 'schoolURL': 'http://www.schoolfinds.com/spring-field-public-school.html', '_id': ObjectId('5b89c5f6edf2ee1498acd47d')}

School Inserted with success
School Details :  {'schoolName': 'SOPHIA CONVENT SCHOOL', 'schoolAddress': ': AMBALA ROAD CATHOLIC CHURCH KALKA HARYANA -133302', 'schoolEmail': 'Email: principal@bright-scholar.com', 'schoolContact': 'Contact Numbe

School Inserted with success
School Details :  {'schoolName': 'JAINENDRA PUBLIC SCHOOL', 'schoolAddress': ': SECTOR-1- PANCHKULA DISTT AMBALA 134108 HARYANA -134108', 'schoolEmail': 'Email: jpsprincipal@yahoo.co.in', 'schoolContact': 'Contact Number: 0172 -2584571', 'schoolURL': 'http://www.schoolfinds.com/jainendra-public-school.html', '_id': ObjectId('5b89c633edf2ee1498acd492')}

School Inserted with success
School Details :  {'schoolName': 'INDIAN PUBLIC SCHOOL', 'schoolAddress': ': MOHRA DISTT AMBALA HARYANA -133004', 'schoolEmail': 'Email: ips_mohra@yahoo.co.in', 'schoolContact': 'Contact Number: 0171 -2008226-27', 'schoolURL': 'http://www.schoolfinds.com/indian-public-school-11006.html', '_id': ObjectId('5b89c636edf2ee1498acd493')}

School Inserted with success
School Details :  {'schoolName': 'GURU HARKISHAN PUBLIC SCHOOL', 'schoolAddress': ': VILL SAHA JAGADHRI ROAD AMBALA CANTT HARYANA -133104', 'schoolEmail': 'Email: harkishanmemial@yahoo.com', 'schoolContact': 'Contact Numbe